In [16]:
using Revise, DataFrames, NIRBeer, JDF, Arrow, CSV, Plots, FiniteDifferences, ForwardDiff, Jchemo

Load the data

In [7]:
df_dsa = disallowmissing(
    DataFrame(
        JDF.load("data/Beer_Analyzer_Data_clean.jdf")
        )
);


df_nir = disallowmissing(
    DataFrame(
        JDF.load("data/Fermentation_NIR_absorbance_full.jdf")
        )
);

In [ ]:
model = snv() 
#model = snv(scal = false) 
fit!(model, Xtrain)

In [86]:
# Make matrix of NIR spectra row 4:end


batch_names = unique(df_nir.BatchName)


df = []
for b in batch_names
    # Filter df for individiual fermentations
    batch = filter(row -> row.BatchName == b, df_nir)

    X = Matrix(batch[:, 34:end-30])

    model = snv() 

    fit!(model, X)

    X = transf(model, X)

    n = outlier_detection(X; alpha=0.1, max_drop_frac=0.05, max_iter=3)

    push!(df, batch[n[1], :])

end


combined_df = reduce(vcat, df)

Reaching max_drop_frac at iteration 1, stopping.
Reaching max_drop_frac at iteration 1, stopping.
Reaching max_drop_frac at iteration 1, stopping.
Reaching max_drop_frac at iteration 1, stopping.
Reaching max_drop_frac at iteration 1, stopping.
Reaching max_drop_frac at iteration 1, stopping.


Row,TimeStamp,CumulativeTime,BatchName,WL1,WL2,WL3,WL4,WL5,WL6,WL7,WL8,WL9,WL10,WL11,WL12,WL13,WL14,WL15,WL16,WL17,WL18,WL19,WL20,WL21,WL22,WL23,WL24,WL25,WL26,WL27,WL28,WL29,WL30,WL31,WL32,WL33,WL34,WL35,WL36,WL37,WL38,WL39,WL40,WL41,WL42,WL43,WL44,WL45,WL46,WL47,WL48,WL49,WL50,WL51,WL52,WL53,WL54,WL55,WL56,WL57,WL58,WL59,WL60,WL61,WL62,WL63,WL64,WL65,WL66,WL67,WL68,WL69,WL70,WL71,WL72,WL73,WL74,WL75,WL76,WL77,WL78,WL79,WL80,WL81,WL82,WL83,WL84,WL85,WL86,WL87,WL88,WL89,WL90,WL91,WL92,WL93,WL94,WL95,WL96,WL97,⋯
,DateTime,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,2024-02-22T19:34:54,0.0,01_Zwickelbier,1.34102,1.45533,1.6108,2.00854,1.23095,1.4425,1.56024,1.52893,1.97015,2.32902,1.85779,2.27088,1.88423,1.92845,1.64122,1.76808,1.69524,1.84448,1.80291,1.86227,1.85044,1.80803,1.88827,1.83039,1.86196,1.9107,1.94287,1.92382,1.89357,1.8988,1.9203,1.92578,1.90601,1.9188,1.90432,1.92155,1.9013,1.89645,1.91699,1.89748,1.9296,1.91987,1.92694,1.92666,1.9326,1.91576,1.92623,1.92292,1.91478,1.93089,1.92322,1.92807,1.93243,1.91538,1.92852,1.93269,1.93626,1.93024,1.93041,1.93451,1.93167,1.92362,1.92892,1.927,1.92959,1.92994,1.93153,1.91771,1.92928,1.93929,1.91942,1.92457,1.93165,1.92858,1.93096,1.93417,1.9278,1.93178,1.94339,1.95066,1.93799,1.93818,1.93846,1.94715,1.93666,1.93999,1.94227,1.93096,1.90545,1.85225,1.7886,1.73472,1.6985,1.67859,1.67151,1.67044,1.66725,⋯
2,2024-02-22T19:37:54,0.05,01_Zwickelbier,1.24404,1.295,1.73238,1.39359,1.68893,1.48785,1.72596,1.30166,1.71684,2.12247,1.87611,1.74131,1.70129,1.76445,1.5978,1.71903,1.8076,1.80848,1.80605,1.84315,1.78023,1.85834,1.81224,1.86983,1.8645,1.84709,1.90826,1.91733,1.87105,1.92224,1.88325,1.87069,1.89975,1.90204,1.90356,1.92633,1.88194,1.89472,1.91603,1.90247,1.9051,1.90693,1.90614,1.93004,1.91006,1.9191,1.90191,1.90109,1.90282,1.90644,1.91333,1.90329,1.9128,1.8999,1.90509,1.91519,1.9146,1.91138,1.91573,1.91071,1.91096,1.90945,1.91498,1.90795,1.90661,1.90523,1.90075,1.90148,1.89383,1.90016,1.90679,1.9076,1.89718,1.91393,1.90568,1.90663,1.89903,1.90836,1.90661,1.91804,1.91994,1.91383,1.9144,1.92482,1.91068,1.92281,1.92909,1.91336,1.88997,1.83374,1.7732,1.71663,1.67789,1.65742,1.65505,1.65284,1.6496,⋯
3,2024-02-22T19:40:54,0.1,01_Zwickelbier,1.33411,2.03789,1.50025,1.70582,1.40409,1.3577,1.44401,1.95321,1.61635,1.9034,1.92347,1.75414,1.5603,1.71408,1.61489,1.67121,1.66472,1.76322,1.79708,1.8104,1.78405,1.7625,1.83956,1.84015,1.83942,1.83346,1.8744,1.92672,1.8974,1.84016,1.89294,1.86162,1.85319,1.88734,1.84478,1.8661,1.87568,1.84812,1.85725,1.86644,1.86394,1.85708,1.86437,1.8713,1.87181,1.87099,1.87481,1.86966,1.87616,1.86789,1.88393,1.88065,1.86698,1.87869,1.87428,1.87454,1.87145,1.87404,1.87507,1.86934,1.86497,1.8668,1.86886,1.87181,1.86098,1.85772,1.86528,1.86426,1.85945,1.85677,1.86443,1.85786,1.85705,1.86515,1.85864,1.85066,1.85469,1.85924,1.86507,1.8643,1.87165,1.87006,1.86975,1.86502,1.86526,1.87312,1.87026,1.86293,1.83564,1.77727,1.71477,1.66494,1.62538,1.60236,1.59316,1.59537,1.59136,⋯
4,2024-02-22T19:43:54,0.15,01_Zwickelbier,1.27079,1.89588,1.41148,1.29009,1.25902,1.28265,1.67936,1.71666,1.5864,1.66812,1.90676,1.65817,1.69318,1.84574,1.74833,1.80095,1.83231,1.75512,1.82575,1.81007,1.8341,1.87081,1.85128,1.85955,1.86469,1.87362,1.90044,1.896

In [ ]:
min_time = 1
max_time = 12
k_rate = 20
exponent = 3/2

# Get unique batch names
batch_names = disallowmissing(unique(combined_df.BatchName))
idx_selected = []
for batch in batch_names[2:2]
    itp = pchip_interpolation(df_dsa, batch, :extract)
    last_sample = last(df_dsa[df_dsa.BatchName .== batch, :CumulativeTime])
    cumulative_time = combined_df[combined_df.BatchName .== batch, :CumulativeTime]
    cumulative_time = cumulative_time[cumulative_time .<= last_sample]

    
    last_time = -Inf

    for (i, t) in enumerate(cumulative_time)
        dE = abs(ForwardDiff.derivative(itp, t))

        dt_local =  max_time * exp(-k_rate * dE)^exponent
        dt_local = max(dt_local, min_time)

        if t >= last_time + dt_local
            push!(idx_selected, i)
            last_time = t
        end
       
    end
    println("Batch: $batch, selected points: $(length(idx_selected))")
    break
end

Batch: 02_Zwickelbier, selected points: 68


In [99]:
idx_selected

68-element Vector{Any}:
    1
  178
  335
  455
  555
  642
  721
  795
  866
  935
    ⋮
 4914
 5102
 5292
 5482
 5667
 5841
 6030
 6231
 6439

In [98]:
batch = "02_Zwickelbier"

last_sample = last(df_dsa[df_dsa.BatchName .== batch, :CumulativeTime])
println("Last sample time for batch $batch: $last_sample h")

itp = pchip_interpolation(df_dsa, batch, :extract)
ts = combined_df[combined_df.BatchName .== batch, :CumulativeTime]
ts_selected = ts[ts .<= last_sample]
ys = itp.(ts_selected)

scatter_idx = 1:length(ts_selected)

plot(ts_selected, ys, label="Interpolation", xlabel="Cumulative Time / h", ylabel="Extract / °P")
scatter!(ts_selected[idx_selected], ys[idx_selected], label="Selected Points", color=:red, markershape=:cross, markersize=3)

Last sample time for batch 02_Zwickelbier: 326.75 h


BoundsError: BoundsError: attempt to access 2640-element Vector{Float64} at index [Any[1, 178, 335, 455, 555, 642, 721, 795, 866, 935, 1003, 1072, 1143, 1217, 1294, 1370, 1444, 1515, 1583, 1647, 1707, 1763, 1813, 1859, 1902, 1944, 1985, 2025, 2066, 2107, 2150, 2196, 2246, 2299, 2354, 2410, 2468, 2528, 2589, 2651, 2714, 2776, 2838, 2901, 2967, 3038, 3117, 3204, 3295, 3393, 3502, 3630, 3767, 3911, 4043, 4202, 4368, 4541, 4725, 4914, 5102, 5292, 5482, 5667, 5841, 6030, 6231, 6439]]